<a href="https://colab.research.google.com/github/pcmbs/preset-embedding_audio-model-selection/blob/main/notebooks_colab/preset_embedding_audio_model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone GitHub repo

In [1]:
!git clone -b main https://github.com/pcmbs/preset-embedding_audio-model-selection.git

# change current directory
%cd /content/preset-embedding_audio-model-selection

Cloning into 'preset-embedding_audio-model-selection'...
remote: Enumerating objects: 827, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 827 (delta 38), reused 40 (delta 15), pack-reused 749
Receiving objects: 100% (827/827), 104.92 MiB | 22.94 MiB/s, done.
Resolving deltas: 100% (350/350), done.
/content/preset-embedding_audio-model-selection


Download dependencies from requirements.txt

In [2]:
#@title
!pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.3 MB/s eta 0:00:00

set PROJECT_ROOT python environment variable (for hydra config)

In [3]:
import os
os.environ["PROJECT_ROOT"] = "/content/preset-embedding_audio-model-selection"

# Download nsynth-lite
(only required for the nearest neighbors evaluation)

In [ ]:
#@title
!cd data && gdown "1720jJyIB1BAVoGg5Caq1SF78Gy4vZuEt" && unzip -q -o nsynth-lite.zip && rm nsynth-lite.zip && rm -r __MACOSX

Downloading...
From: https://drive.google.com/uc?id=1720jJyIB1BAVoGg5Caq1SF78Gy4vZuEt
To: /content/preset-embedding_audio-model-colab/data/nsynth-lite.zip
100% 1.65G/1.65G [00:15<00:00, 103MB/s]


# Download sound attributes ranking dataset
(only required for the sound attributes ranking evaluation)


In [4]:
!cd data/TAL-NoiseMaker && gdown "12L80oXwX9NkD1witr0-s3ed1txQ8zA-e" && unzip -q -o sound_attributes_ranking_dataset_mono.zip && rm sound_attributes_ranking_dataset_mono.zip

Downloading...
From: https://drive.google.com/uc?id=12L80oXwX9NkD1witr0-s3ed1txQ8zA-e
To: /content/preset-embedding_audio-model-selection/data/TAL-NoiseMaker/sound_attributes_ranking_dataset_mono.zip
100% 745M/745M [00:05<00:00, 143MB/s]


# Hydra Config

Run the following cell to print the current config

In [5]:
#@title
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.hydra_config import HydraConfig

with initialize(version_base=None, config_path="configs", job_name="print_cfg"):
    cfg = compose(config_name="evaluate_audio_model", return_hydra_config=True)
    cfg.hydra.job.num = 0 # arbitrary value, otherwise raise an error since missing
    cfg.hydra.runtime.output_dir = cfg.hydra.run.dir # same as above, should be that for single run
    HydraConfig.instance().set_config(cfg)
    cfg_no_hydra = {k: v for k, v in cfg.items() if k != "hydra"} # dont print hydra config
    print(OmegaConf.to_yaml(cfg_no_hydra, resolve=True))


seed: 667
audio_length: 4.0
task_name: encodec_48khz
wandb:
  dir: /content/preset-embedding_audio-model-selection/logs/encodec_48khz/2023-07-26_15-48-58
  mode: online
  id: null
  anonymous: null
  project: preset-embedding_audio-model-selection
  group: encodec_48khz
  tags: []
model:
  encoder:
    _target_: models.encodec.encoder.EncodecEncoder.encodec_model_48khz
    repository: /content/preset-embedding_audio-model-selection/checkpoints
    segment: 4.0
    overlap: 0.0
  name: encodec_48khz
eval:
  sound_attributes_ranking:
    root: /content/preset-embedding_audio-model-selection/data/TAL-NoiseMaker/sound_attributes_ranking_dataset
distance_fn: pairwise_manhattan_distance
reduce_fn: flatten
paths:
  root_dir: /content/preset-embedding_audio-model-selection
  data_dir: /content/preset-embedding_audio-model-selection/data
  log_dir: /content/preset-embedding_audio-model-selection/logs
  ckpt_dir: /content/preset-embedding_audio-model-selection/checkpoints
  output_dir: /content/

# Evaluate audio models

evaluation using the current config

In [ ]:
!python src/evaluate_audio_model.py

evaluate all models, for all reduction functions, for all distance metrics

In [6]:
!python src/evaluate_audio_model.py -m model="glob(*)" distance_fn="glob(*)" reduce_fn="glob(*)"

[2023-07-26 15:49:21,551][HYDRA] Launching 30 jobs locally
[2023-07-26 15:49:21,551][HYDRA] 	#0 : model=encodec24khz distance_fn=pairwise_cosine_similarity reduce_fn=flatten
Global seed set to 667
[2023-07-26 15:49:21,715][__main__][INFO] - Instantiating model <models.encodec.encoder.EncodecEncoder.encodec_model_24khz>
[2023-07-26 15:49:22,300][__main__][INFO] - Instantiating wandb logger
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 2
wandb: You chose 'Use an existing W&B account'
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Tracking run with wandb version 0.15.7
wandb: Run data is saved locally in /c